# 概要
- スクレイピングで対象ページの記事を全てdbに落とす
-  とりあえずdb (id, category, text, url)で定義する

# ライブラリimport 

In [1]:
import requests
import urllib
import sqlite3
import re
import time
from bs4 import BeautifulSoup as BS

# スクレイピング用の関数定義

In [12]:
# トップページからカテゴリ別に分けられてるページのURLを抽出
def main_url_func(main_url):
    main_url_dic = {}
    res = urllib.request.urlopen(main_url)
    data = res.read()
    text = data.decode("utf-8")
    soup = BS(text, 'html.parser')
    ul = soup.html.body.nav.ul
    li = ul.find_all('li', class_=re.compile(r'^(nav_color_).$'),)

    for i in li:    
        main_url_dic[i.a.string] = i.a.attrs['href']
    return main_url_dic


# カテゴリごとのページを訪れ、次のページへのリンクを抽出
def same_category_other_page_url(main_dic):

    for category, start_url  in main_dic.items():
        other_pages_list = [start_url]
        all_url_dic = {}
        
        url = start_url
        res = urllib.request.urlopen(url)
        time.sleep(1)
        data = res.read()
        text = data.decode("utf-8")
        soup = BS(text, "html.parser")
    
        body = soup.html.body
        li = body.find_all("a", href=re.compile(r'^/categories/1.'))
    
        for i in li:
            other_pages_list.append("https://gunosy.com" + i.attrs['href'])
            
        all_url_dic[category] = set(other_pages_list)
        
    return all_url_dic


#　カテゴリ別のページにアクセスして記事のあるURLを抽出
def categpry_url_func(main_dic):
    sub_url_dic = {}
    
    for category, url in main_dic.items():
        url_list = []  
        res = urllib.request.urlopen(url)
        time.sleep(1)
        data = res.read()
        text = data.decode("utf-8")
        soup = BS(text, "html.parser")
        soup_html = soup.html.body
        li = soup_html.find_all("a", href=re.compile(r'^https://gunosy.com/articles/'))
    
        for i in li:
            url_list.append(i.attrs['href'])
        
        sub_url_dic[category] = url_list
    
    return sub_url_dic

# 記事のtextをスクレイピング
#　URLを入力したら、textがリストで返る
def text_scraping(url):
    text_list = []
    res = urllib.request.urlopen(url)
    time.sleep(1)
    data = res.read()
    text = data.decode("utf-8")
    soup = BS(text, "html.parser")
    body = soup.html.body
    articlie = body.find_all("p")
    for i in articlie:
        text_list.append(i.string)
    return text_list

# URLの取得

In [3]:
top_url = "https://gunosy.com"
main_url_dic = main_url_func(top_url)

categpry_url_dic = categpry_url_func(main_url_dic)

In [4]:
categpry_url_dic.keys()

dict_keys(['エンタメ', 'スポーツ', 'おもしろ', '国内', '海外', 'コラム', 'IT・科学', 'グルメ'])

# dbの定義

In [6]:
# dbに接続する
conn = sqlite3.connect('GunosyText.db')
c = conn.cursor()

# テーブルの作成
c.execute('''CREATE TABLE textlist(id real, category real, airticle text, url text)''')

# 記事のスクレイピングと記事をdbに保存
**「逐次的にアクセスして保存」**
1. 記事のURLにアクセス
2. 記事をスクレイピング
3. 　dbに保存

In [7]:
count = 0
categpry_dic = {}
for name in categpry_url_dic.keys():
    categpry_dic[name] = count
    count += 1
categpry_dic

{'エンタメ': 0,
 'スポーツ': 1,
 'おもしろ': 2,
 '国内': 3,
 '海外': 4,
 'コラム': 5,
 'IT・科学': 6,
 'グルメ': 7}

In [13]:
# (id, category, text, url)
index = 0

for name, url_list in categpry_url_dic.items():
    
    for url in url_list:
        text = text_scraping(url)
        print(url + "TextList_compreted")
        c.execute("INSERT INTO textlist VALUES(index, categpry_dic[name], text, url)")
        index += 1
    print(name + "completed" + "-"*20)

https://gunosy.com/articles/awjBhTextList_compreted


OperationalError: near "index": syntax error